In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from typing import Optional, Callable
import os

from scripts.loops import train_loop, valid_loop, test_loop
from scripts.dataset_template import DatasetTemplate
from scripts.models.autoencoder_cnn import CNNAutoencoder
from scripts.models.autoencoder_lin import LinearAutoencoder
from scripts.models.autoencoder_lstm import LSTMAutoencoder

In [2]:
#check if cuda available

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [3]:
#paths
data_train_path = "/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/Data/Data_channels/train/comeretial"
data_test_folder_path = "/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/Data/Data_channels/test"
data_valid_folder_path = "/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/Data/Data_channels/valid"

testing_folders = os.listdir(data_test_folder_path)
valid_folders = os.listdir(data_valid_folder_path)

In [4]:
#hyperparameters
batch_size = 1
learning_rate = 0.01
epochs = 1

In [5]:
#transforms
data_transforms = transforms.Compose([transforms.ToTensor()])

In [6]:
#datasets and dataloaders

#train
train_dataset = DatasetTemplate(data_train_path, 0, transform = data_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#valid
valid_datasets = {}
valid_dataloaders = {}

for valid_folder_name in valid_folders:
    valid_folder_path_all = os.path.join(data_valid_folder_path, valid_folder_name)
    valid_datasets[valid_folder_name] = DatasetTemplate(valid_folder_path_all, 1)
    valid_dataloaders[valid_folder_name] = DataLoader(valid_datasets[valid_folder_name], batch_size=batch_size, shuffle=True)

#test
test_datasets = {}
test_dataloaders = {}

for test_folder_name in testing_folders:
    test_folder_path_all = os.path.join(data_test_folder_path, test_folder_name)
    test_datasets[test_folder_name] = DatasetTemplate(test_folder_path_all, 1)
    test_dataloaders[test_folder_name] = DataLoader(test_datasets[test_folder_name], batch_size=batch_size, shuffle=True)

In [7]:
valid_dataloaders

{'Fake_Bts_PCI_466_wPA': <torch.utils.data.dataloader.DataLoader at 0x723c24299cc0>,
 'Fake_Bts_PCI_466': <torch.utils.data.dataloader.DataLoader at 0x723c24298940>}

In [8]:
#define model
model = CNNAutoencoder().to(device)
criterion = nn.MSELoss()
loss_RMS = lambda x,y: torch.sqrt(criterion(x, y))
metric_fn_RMS = lambda x,y: torch.sqrt(criterion(x, y))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


valid_loop(test_dataloaders[list(test_dataloaders.keys())[0]],
          model,
          loss_RMS,
          metric_fn_RMS,
          device=device)

valid Error: 
 Accuracy: 327.3862238611494%, Avg loss: 327.386224 



(327.3862238611494, 327.3862238611494)

In [9]:
#epochs

# Lists to log the losses during the training process
train_losses = []
train_metrics = []

valid_losses_true = []
valid_metrics_true = []

valid_losses_false = []
valid_metrics_false = []

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loss, train_metric = train_loop(train_dataloader,
                                     model,
                                     loss_RMS,
                                     optimizer)
    
  valid_loss_true, valid_metric_true = valid_loop(valid_dataloaders['Fake_Bts_PCI_466'],
                                                  model,
                                                  loss_RMS,
                                                  metric_fn_RMS)

  valid_loss_false, valid_metric_false = valid_loop(valid_dataloaders['Fake_Bts_PCI_466_wPA'],
                                                    model,
                                                    loss_RMS,
                                                    metric_fn_RMS)
    

  train_losses.append(train_loss)
  train_metrics.append(train_metric)

  valid_losses_true.append(valid_loss_true)
  valid_metrics_true.append(valid_metric_true)

  valid_losses_false.append(valid_loss_false)
  valid_metrics_false.append(valid_metric_false)
    

print("Done!")

Epoch 1
-------------------------------
loss: 243.277802  [    0/81710]

loss: 238.431961  [10000/81710]

loss: 164.113754  [20000/81710]

loss: 223.522156  [30000/81710]

loss: 86.036797  [40000/81710]

loss: 381.961426  [50000/81710]

loss: 115.885506  [60000/81710]

loss: 129.367615  [70000/81710]

loss: 54.749916  [80000/81710]

valid Error: 
 Accuracy: 708.2587492752075%, Avg loss: 708.258749 

valid Error: 
 Accuracy: 563.1159388911617%, Avg loss: 563.115939 

Done!


In [15]:
#testing

for testing_name, testing_loader in test_dataloaders.items():
    test_losses, test_metrics = test_loop(testing_loader, 
                                            model,
                                            loss_RMS,
                                             metric_fn_RMS)
    mean_loss = np.mean(np.array(test_losses))
    mean_metrics = np.mean(np.array(test_metrics))
    print(f"Dataset name: {testing_name}, mean_loss: {mean_loss}, mean_acc: {mean_metrics}")

Dataset name: Fake_Bts_PCI_466_wPA_traffic, mean_loss: 327.39576672534554, mean_acc: 0
Dataset name: Fake_Bts_PCI_12, mean_loss: 448.1440302961964, mean_acc: 0
Dataset name: Fake_Bts_PCI_466_traffic, mean_loss: 922.3979322760891, mean_acc: 0
Dataset name: Fake_Bts_PCI_12_wPA, mean_loss: 537.0707053340338, mean_acc: 0


In [13]:
test_losses

[889.3809814453125,
 296.5823974609375,
 106.67228698730469,
 122.99565124511719,
 118.70726776123047,
 328.84014892578125,
 189.2257080078125,
 220.09603881835938,
 350.0352478027344,
 180.5941619873047,
 529.2520751953125,
 377.1189880371094,
 180.3527374267578,
 395.30706787109375,
 102.52214813232422,
 119.42941284179688,
 503.606201171875,
 207.30592346191406,
 190.8264923095703,
 288.6839294433594,
 352.7959289550781,
 659.2632446289062,
 598.281982421875,
 190.7733917236328,
 70.51310729980469,
 112.93157196044922,
 400.6371765136719,
 502.2035217285156,
 166.1306915283203,
 237.534912109375,
 252.37991333007812,
 286.4708251953125,
 988.7996215820312,
 489.1368103027344,
 360.4900207519531,
 211.71231079101562,
 205.2598419189453,
 456.5553283691406,
 108.9272232055664,
 220.6190643310547,
 174.19403076171875,
 390.5556335449219,
 407.4954833984375,
 172.98248291015625,
 179.07981872558594,
 464.94256591796875,
 396.3542175292969,
 1024.301025390625,
 237.64573669433594,
 110.2